# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [2]:
# Your code here; import the necessary packages

import sqlite3
import pandas as pd

In [3]:
# Your code here; create the connection

conn = sqlite3.connect("data.sqlite")
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [5]:
# Your code here

query_1 = """
SELECT customerNumber, contactLastName, contactFirstName
FROM customers
WHERE customerNumber IN (
    SELECT customerNumber 
    FROM orders 
    WHERE orderDate = '2003-01-31'
);
"""

df = pd.read_sql_query(query_1, conn)
print(df)

   customerNumber contactLastName contactFirstName
0             141          Freyre           Diego 


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [7]:
# Your code here

query_2 = """
SELECT 
    p.productName, 
    SUM(od.quantityOrdered) AS total_items_sold
FROM orderdetails od
JOIN products p 
    ON od.productCode = p.productCode
GROUP BY p.productName
ORDER BY total_items_sold DESC;
"""

df = pd.read_sql_query(query_2, conn)
print(df)

                                 productName  total_items_sold
0                1992 Ferrari 360 Spider red              1808
1                       1937 Lincoln Berline              1111
2                  American Airlines: MD-11S              1085
3    1941 Chevrolet Special Deluxe Cabriolet              1076
4               1930 Buick Marquette Phaeton              1074
..                                       ...               ...
104             1999 Indy 500 Monte Carlo SS               855
105                       1911 Ford Town Car               832
106         1936 Mercedes Benz 500k Roadster               824
107               1970 Chevy Chevelle SS 454               803
108                    1957 Ford Thunderbird               767

[109 rows x 2 columns]


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [11]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

query_3= """
SELECT 
    p.productName, 
    COUNT(DISTINCT o.customerNumber) AS total_customers
FROM orderdetails od
JOIN orders o 
    ON od.orderNumber = o.orderNumber
JOIN products p 
    ON od.productCode = p.productCode
GROUP BY p.productName
ORDER BY total_customers DESC;
"""

df = pd.read_sql_query(query_3, conn)
print(df)

                             productName  total_customers
0            1992 Ferrari 360 Spider red               40
1                       Boeing X-32A JSF               27
2                    1972 Alfa Romeo GTA               27
3               1952 Alpine Renault 1300               27
4                     1934 Ford V8 Coupe               27
..                                   ...              ...
104  1958 Chevy Corvette Limited Edition               19
105                  2002 Chevy Corvette               18
106            1969 Chevrolet Camaro Z28               18
107                    1952 Citroen-15CV               18
108                   1949 Jaguar XK 120               18

[109 rows x 2 columns]


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [13]:
# Your code here

query_4 = """
SELECT DISTINCT e.employeeNumber, e.firstName, e.lastName, o.city, o.officeCode
FROM employees e
JOIN offices o 
    ON e.officeCode = o.officeCode
JOIN customers c 
    ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders od 
    ON c.customerNumber = od.customerNumber
WHERE c.customerNumber IN (
    SELECT customerNumber
    FROM orders
    WHERE orderNumber IN (
        SELECT orderNumber
        FROM orderdetails
        WHERE productCode IN (
            SELECT productCode
            FROM (
                SELECT productCode, COUNT(DISTINCT customerNumber) AS customer_count
                FROM orderdetails
                JOIN orders ON orderdetails.orderNumber = orders.orderNumber
                GROUP BY productCode
                HAVING customer_count < 20
            )
        )
    )
);
"""

df = pd.read_sql_query(query_4, conn)
print(df)

    employeeNumber firstName   lastName           city  officeCode
0             1166    Leslie   Thompson  San Francisco           1
1             1370    Gerard  Hernandez          Paris           4
2             1165    Leslie   Jennings  San Francisco           1
3             1323    George     Vanauf            NYC           3
4             1337      Loui     Bondur          Paris           4
5             1621      Mami      Nishi          Tokyo           5
6             1286  Foon Yue      Tseng            NYC           3
7             1216     Steve  Patterson         Boston           2
8             1612     Peter      Marsh         Sydney           6
9             1501     Larry       Bott         London           7
10            1401    Pamela   Castillo          Paris           4
11            1611      Andy     Fixter         Sydney           6
12            1504     Barry      Jones         London           7
13            1702    Martin     Gerard          Paris        

## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [15]:
# Your code here

query_5 = """
SELECT 
    e.employeeNumber, 
    e.firstName, 
    e.lastName, 
    COUNT(c.customerNumber) AS num_customers
FROM employees e
JOIN customers c 
    ON e.employeeNumber = c.salesRepEmployeeNumber
WHERE e.employeeNumber IN (
    SELECT salesRepEmployeeNumber
    FROM customers
    GROUP BY salesRepEmployeeNumber
    HAVING AVG(creditLimit) > 15000
)
GROUP BY e.employeeNumber, e.firstName, e.lastName;
"""

df = pd.read_sql_query(query_5, conn)
print(df)

    employeeNumber firstName   lastName  num_customers
0             1165    Leslie   Jennings              6
1             1166    Leslie   Thompson              6
2             1188     Julie   Firrelli              6
3             1216     Steve  Patterson              6
4             1286  Foon Yue      Tseng              7
5             1323    George     Vanauf              8
6             1337      Loui     Bondur              6
7             1370    Gerard  Hernandez              7
8             1401    Pamela   Castillo             10
9             1501     Larry       Bott              8
10            1504     Barry      Jones              9
11            1611      Andy     Fixter              5
12            1612     Peter      Marsh              5
13            1621      Mami      Nishi              5
14            1702    Martin     Gerard              6


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!